# Mexico COVID-19 clinical data
<p>This daset contains the results of real-time PCR testing for COVID-19 from the [General Directorate of Epidemiology](https://www.gob.mx/salud/documentos/datos-abiertos-152127) (more details [here](href="https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico/resource/e8c7079c-dc2a-4b6e-8035-08042ed37165)).The data was gathered using a "sentinel model" that samples 10% of the patients that present a viral respiratory diagnosis to test for COVID-19, and consists of data reported by 475 viral respiratory disease monitoring units (hospitals) named USMER (Unidades Monitoras de Enfermedad Respiratoria Viral) throughout the country in the entire health sector (IMSS, ISSSTE, SEDENA, SEMAR, and others).</p>

### 1. Load the data 📈

In [ ]:
import pandas as pd
import numpy as np
import warnings

pd.set_option('display.max_columns', None) # shows all columns
warnings.filterwarnings('ignore')

In [ ]:
# Read file into memory

# It's best to try first with the lastest remote file from repository
data = pd.read_csv('https://raw.githubusercontent.com/marianarf/covid19_mexico_analysis/master/mexico_covid19.csv')

# Local data
# data = pd.read_csv('../input/mexico-covid19-clinical-data/mexico_covid19.csv')

In [ ]:
# Show first 5 rows
data.head(5)

In [ ]:
# Quick glance at the data
data.describe()

### 2. Features 🖼️
<p>We are aiming to predict whether a patient with pending COVID-19 results will get a positive or a negative result. We will use columns already factored with numeric values and will remove variables not relevant to the analysis.<p>

In [ ]:
# Make a copy of the data frame so that we don't override the original dataframe
df = data.copy()

In [ ]:
# Take a glance at the variables
df.keys()

In [ ]:
# Check dtypes for each column
# df.dtypes

In [ ]:
# Exclude features as described above
df = df[df.columns[~df.columns.isin(
    ['id', 'ID_REGISTRO',
     'FECHA_ARCHIVO', 'FECHA_ACTUALIZACION', 'FECHA_DEF', 'FECHA_INGRESO','FECHA_SINTOMAS',
     'ABR_ENT', 'ENTIDAD', 'MIGRANTE', 'NACIONALIDAD', 'ORIGEN', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN',
     'INTUBADO', 'UCI'] # remove features that are only available while hospitalized
)]]

In [ ]:
# Show the number of missing (NAN, NaN, na) data for each column
# data.isnull().sum()

In [ ]:
# Various ways to check for NaN, NA and NULL
# df.isnull()
# df.isnull().sum()
# df.isnull().values.any()
# df.isnull().values.sum()
# df.isnull().any()

In [ ]:
# There are a few rows without the city code attribute - so let's remove them
df = df[~df.isnull().any(axis=1)]

In [ ]:
# We have data that contains either negative or positive results (i.e, excludes tests that are in process)
print(df['RESULTADO'].unique())

In [ ]:
# The original data has different codes, but let's follow convention and refactor them (0=negative, 1=positive)
df['RESULTADO'] = df['RESULTADO'].astype(str).str.replace('2','0') # negative
df['RESULTADO'] = df['RESULTADO'].astype(str).str.replace('1','1') # positive

In [ ]:
# Convert whole df to numeric
df = df.apply(pd.to_numeric, errors='ignore')

In [ ]:
# We want every remaining column to be of numeric type
df.info()

### 3. Model 🤖
<p>We are aiming to predict whether a patient with pending COVID-19 results will get a positive or a negative result:</p>

<ul>
<li>As laboratory results are processed, each pending patient record leaves a time window when it's uncertain whether a result will return positive or negative (so we can validate almost daily if predictions are wrong or right, as laboratory results are reported in the new reports).</li>
<li>Also, this could help predict for similar symptoms e.g. from a survey or an app that checks for similar data (ideally, containing most of the parameters that can be assessed without coming into the hospital, like e.g. age of the patient)</li>
</ul>

<p>The value of the lab result comes from a RT-PCR, and is stored in <code>RESULTADO</code>, where <code>0 = NEGATIVE</code> and <code>1 = POSITIVE</code>. Let's rename this to <code>target</code> so that it's more convenient to work with.</p>

In [ ]:
# Rename target column as 'target' for clarity
df.rename(
    columns={'RESULTADO': 'target'},
    inplace=True
)

In [ ]:
# Remove target variable to move it to the first position of dataframe
col_name = 'target'
first_col = df.pop(col_name)

In [ ]:
# Now we can use Pandas insert() function and insert the opped column into first position of the dataframe
# The first argument of insert() function is the location we want to insert, here it is 0
df.insert(0, col_name, first_col)

In [ ]:
# Now response variable is at the start of the data frame
df.head()

In [ ]:
# Let's see how many observations and features we have
df.shape

### 3.1 Explore features 🔬

### 3.1.1 Correlation Matrix 📐

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

corrmat = df.corr() 
  
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax=ax, cmap='viridis', linewidths = 0.1)

> ### 3.1.2 Target incidence 🎯
<p>Target incidence is the number of cases of each individual target value in the data. It represents the proportion of categorical variable in a target column. Target incidence gives us an idea of how balanced (imbalanced) is our data.</p>

In [ ]:
# Print target incidence proportions and round to 3 decimal places
df.target.value_counts(normalize=True).round(3)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (15,6))

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16

five_thirty_eight = [
    '#30a2da',
    '#fc4f30',
    '#e5ae38',
    '#6d904f',
    '#8b8b8b',
]

sns.set_palette(sns.color_palette(five_thirty_eight))

total = float(len(data))
results = ['Negative', 'Positive']  
ax = sns.countplot(x='target', data=df)
ax.set_xticklabels(results)

# add percentages above each bar
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            p.get_height(),
            '{:1.2f}'.format(height/total),
            ha='center')
    
plt.title('Incidence of COVID-19 in Mexico', fontsize=20)
plt.xlabel('Result')
plt.ylabel('Patients')

sns.set(style='ticks')
sns.despine()

### 3.1.3 Age distribution of cases for each category 🕵️

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_positive = df.loc[df['target'] == 0]
df_negative = df.loc[df['target'] == 1]

five_thirty_eight = [
    '#30a2da',
    '#fc4f30',
    '#e5ae38',
    '#6d904f',
    '#8b8b8b',
]

plt.figure(figsize = (15,6))
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16

sns.set_palette(sns.color_palette(five_thirty_eight))

sns.distplot(df_positive['EDAD'])
sns.distplot(df_negative['EDAD'])

sns.set(style='ticks')
sns.despine()

### 3.1.4 Class imbalance for each feature 📊

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

five_thirty_eight = [
    '#30a2da',
    '#fc4f30',
    '#e5ae38',
    '#6d904f',
    '#8b8b8b',
]
    
var = df.columns.values

i = 0
t0 = df.loc[df['target'] == 0]
t1 = df.loc[df['target'] == 1]

plt.figure()
fig, ax = plt.subplots(8,4,figsize=(16,28))
plt.title('Class imbalance for features in data')

for feature in var:
    i += 1
    plt.subplot(8,4,i)
    sns.kdeplot(t0[feature], bw=0.5, label="Negative = 0")
    sns.kdeplot(t1[feature], bw=0.5, label="Positive = 1")
    sns.set_palette(sns.color_palette(five_thirty_eight))
    sns.set(style='ticks')
    sns.despine()
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
plt.show();

### 3.2 Binary classification
<p>We want to predict whether or not a person with a certain clinical profile may result COVID-19 positive when tested for a real-time PCR. The model for this is a binary classifier, meaning that there are only 2 possible outcomes:</p>

<ul>
    <li><code>0</code> - the lab result is COVID-19 negative</li>
    <li><code>1</code> - the lab result is COVID-19 positive</li>
</ul>

<p>1. As laboratory results are processed, each pending patient record leaves a time window when it's uncertain whether a result will return positive or negative (so we can validate almost daily if predictions are wrong or right, as laboratory results are reported in the new reports).</p>

<p>2. Also, this could help predict for similar symptoms e.g. from a survey or an app that checks for similar data (ideally, containing most of the parameters that can be assessed without coming into the hospital, like e.g. age of the patient).</p>

### 3.2.1 Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('target',axis=1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 3.2.2 Models and hyperparameters
<p>This section is adapted from [this post by Gabriel Naya](https://towardsdatascience.com/available-hyperparameter-optimization-techniques-dc60fb836264) on hyperparameter optimization with the following optimization techniques:<p>
    
<ul>
    <li>Default hyperparameters</li>
    <li>Sklearn GridSearchCV</li>
    <li>Sklearn RandomizedSearchCV</li>
    <li>Hyperopt for Python</li>
</ul>

In [ ]:
# First create dataframe to store the results
cols = ['Case','RndForest','LogReg','LGB']
result_tbl = pd.DataFrame(columns=cols)
result_tbl.set_index('Case',inplace=True)
result_tbl.loc['Standard'] = [0,0,0]
result_tbl.loc['GridSearch'] = [0,0,0]
result_tbl.loc['RandomSearch'] = [0,0,0]
result_tbl.loc['Hyperopt'] = [0,0,0]
result_tbl.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Instantiate models with default parameters

rf    = RandomForestClassifier(n_estimators=10)
lr    =  LogisticRegression(solver='liblinear')
lgg   = lgb.LGBMClassifier()
models = [rf,lr,lgg]

col = 0

for model in models:
    model.fit(X_train,y_train.values.ravel())
    result_tbl.iloc[0,col] = model.score(X_test,y_test)
    col += 1
result_tbl.head()

In [ ]:
# Parameters for GridSearchCV

# Random Forest
n_estimators = [10, 100, 1000,10000]
max_features = ['sqrt', 'log2']
rf_grid = dict(n_estimators=n_estimators, max_features=max_features)

# Logistic Regrresion
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
lr_grid = dict(solver=solvers, penalty=penalty, C=c_values)

# LGB
scale_pos_weight = (161217/103095) # change this to not-manual input -_- pls
boosting_type = ['gbdt', 'goss', 'dart']
num_leaves = [30,50,100,150] #list(range(30, 150)),
learning_rate = list(np.logspace(np.log(0.005), np.log(0.2), base = np.exp(1), num = 10)) #1000
lgg_grid = dict(scale_pos_weight=scale_pos_weight, boosting_type=boosting_type, num_leaves=num_leaves, learning_rate=learning_rate)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

# This configuration depends on one's computing capacity and scope of time.
# The parameters in the previous cell are only some parameters available for the models.
# I haven't figured how to use GPU for the following optimization techniques, so I will use LightXGB.

models = [rf,lr,lgg]
grids = [rf_grid,lr_grid,lgg_grid]

col = 0
for ind in range(0,len(models)):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, 
                                 random_state=1)
    grid_search = GridSearchCV(estimator=models[col], 
                  param_grid=grids[col], n_jobs=-1, cv=cv,  
                  scoring='accuracy',error_score=0)
    grid_clf_acc = grid_search.fit(X_train, y_train)
    result_tbl.iloc[1,col] = grid_clf_acc.score(X_test,y_test)
    col += 1
result_tbl.head()

In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

# This configuration depends on one's computing capacity and scope of time.
# The parameters in the previous cell are only some parameters available for the models.
# I haven't figured how to use GPU for the following optimization techniques, so I will use LightXGB.

models = [rf,lr,lgg]
grids = [rf_grid,lr_grid,lgg_grid]

col = 0
for ind in range(0,len(models)):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, 
                                 random_state=1)
    n_iter_search = 3
    random_search = RandomizedSearchCV(models[col],
    param_distributions=grids[col],n_iter=n_iter_search, cv=cv)
    random_search.fit(X_train,y_train)
    result_tbl.iloc[2,col] = random_search.score(X_test, y_test)
    col += 1
result_tbl.head()

### 4. LightXGB model
<p>Since I couldn't use the GPU and it will take a long time to perform the hyperparameter optimization, I opted to use LightXGB with the base parameters. Imo there's no much use in performing these steps before having a good baseline model with the correct feature engineering but I will leave the code if someone wants to try it out with different algorithms!</p>

### 4.1 Train

In [ ]:
import lightgbm as lgb

# Instantiate model

lgg = lgb.LGBMClassifier()
lgg.fit(X_train,y_train.values.ravel())
print(lgg.score(X_test,y_test))

In [ ]:
# Predict the results
y_hat = lgg.predict(X_test)

### 4.2 Performance metrics

In [ ]:
from sklearn.metrics import accuracy_score

# View accuracy
accuracy = accuracy_score(y_hat, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_hat)))

In [ ]:
# Compare train and test set accuracy
y_hat_train = lgg.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_hat_train)))

In [ ]:
# Check for overfitting
print('Training set score: {:.4f}'.format(lgg.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(lgg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import classification_report

# Classification metrics
print(classification_report(y_test, y_hat))

In [ ]:
from sklearn.metrics import confusion_matrix

# Confussion matrix
cm = confusion_matrix(y_test, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

### 4.3 Too many false negatives! 🤔
<p>Since this model is meant to build for diagnostic, it's very dangerous to have that many false negatives. Let's explore ROC metrics to have a better understanding of what's going on.</p>

In [ ]:
# Calculate probability again
y_hat = lgg.predict_proba(X_test) # Use this instead of `predict()`so that we can retrieve probabilities
pos_probs = y_hat[:,1] # Retrieve just the probabilities for the positive class

### 4.3.1 ROC Curve

In [ ]:
from sklearn.metrics import roc_curve

# Plot No Skill ROC Curve
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')

# Calculate ROC Curve for model
fpr, tpr, _ = roc_curve(y_test, pos_probs)

# Plot model ROC Curve
plt.plot(fpr, tpr, marker='.', label='LightGBM')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

### 4.3.2 ROC AUC Curve

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

# Create No Skill classifier
dummy_model = DummyClassifier(strategy='stratified')
dummy_model.fit(X_train, y_train)
y_hat_dummy = dummy_model.predict_proba(X_test)
pos_probs_dummy = y_hat_dummy[:, 1]

# Calculate ROC AUC for No Skill
roc_auc = roc_auc_score(y_test, pos_probs_dummy)
print('No Skill ROC AUC %.3f' % roc_auc)

# Calculate ROC AUC for our model
roc_auc = roc_auc_score(y_test, pos_probs)
print('LightGBM ROC AUC %.3f' % roc_auc)

### 4.3.3 Precision-Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve

# Calculate the no skill line as the proportion of the positive class
no_skill = len(y[y==1])/len(y)

# Plot the no skill precision-recall curve
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

# Calculate model precision-recall curve
precision, recall, _ = precision_recall_curve(y_test, pos_probs)

# Plot the model precision-recall curve
plt.plot(recall, precision, marker='.', label='LightGBM')

# Calculate model precision-recall curve
precision, recall, _ = precision_recall_curve(y_test, pos_probs)

# Call the plot
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

# Calculate the Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, pos_probs_dummy)
auc_score = auc(recall, precision)
print('No Skill PR AUC: %.3f' % auc_score)

# Calculate the Precision-Recall AUC for our model
precision, recall, _ = precision_recall_curve(y_test, pos_probs)
auc_score = auc(recall, precision)
print('LightGBM PR AUC: %.3f' % auc_score)

### 5. Conclusion and notes for further analysis
<p>We have explored the data, plotted some features and tried a binary classification model. While the preliminary performance scores aren't all that discouraging, this is meant to be only a starting point to expand upon the model. Further analysis may add features most importantly to optimize for the confusion matrix, crucially, in regards for the false negative results, which in this case is at the most center of improvement.</p>

<center>Stay safe and I hope this effort is helpful! 🙋‍♀️🌎</center>